In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [ ]:
path = '../data/'
savepath = '../data/'
#images = glob.glob(path+'new_data/*.npy')
tile_sizes = np.loadtxt(path+'tile_sizes.txt', dtype='int')
tile_sizes = tile_sizes[10:800]
images_sampled = {}
for tile in tile_sizes:
    for i in range(1, 500):
        images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])
print(len(images_sampled.keys()))

In [ ]:
def myGenerator(batch_size):
    s = 192
    while True:
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/nparrs_384/frame'+str(i)+'.npy'
            frame = np.load(frame)
            tile_index = random.choice(images_sampled[i])
            
            tmp = imresize(frame[tile_index], (s, 192))
            temp  = imresize(frame[tile_index], (24, 24))
            temp  = imresize(temp, (s, 192))
            
            alldata_x.append(temp)
            #alldata_y.append(frame[tile_index])
            alldata_y.append(tmp)
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        alldata_x = alldata_x.astype(np.float32)/255.0
        
        yield alldata_x, alldata_y

x = myGenerator(500)
xtrain, ytrain = next(x)
print('xtrain shape:',xtrain.shape)
print('ytrain shape:',ytrain.shape)

In [ ]:
import numpy 
import math
import cv2
import time

def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

data = []
for i in range(5):
    cv2.imwrite(path+'xface' + str(i) + '.jpg', xtrain[i])
    cv2.imwrite(path+'yface' + str(i) + '.jpg', ytrain[i])
    img1 = cv2.imread(path+'xface' + str(i) + '.jpg')
    img2 = cv2.imread(path+'yface' + str(i) + '.jpg')
    d = psnr(img1, img2)
    print (d)

In [ ]:
def psnr2(target, ref):
    # assume RGB image
    target_data = numpy.array(target, dtype=float)
    ref_data = numpy.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(numpy.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

In [ ]:
data = []
for i in range(100):
    img1 = cv2.imread(path+'test/frame_real'+str(i)+'.png')
    img2 = imresize(img1, (1, 1))
    img2 = imresize(img2, (384, 384))
    d = psnr(img1, img2)
    data.append(d)
    print(d)

In [ ]:
"""
Video Quality Metrics
Copyright (c) 2014 Alex Izvorski <aizvorski@gmail.com>

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

import numpy
from scipy.ndimage import gaussian_filter

from numpy.lib.stride_tricks import as_strided as ast

"""
Hat tip: http://stackoverflow.com/a/5078155/1828289
"""
def block_view(A, block=(3, 3)):
    """Provide a 2D block view to 2D array. No error checking made.
    Therefore meaningful (as implemented) only for blocks strictly
    compatible with the shape of A."""
    # simple shape and strides computations may seem at first strange
    # unless one is able to recognize the 'tuple additions' involved ;-)
    shape = (A.shape[0]/ block[0], A.shape[1]/ block[1])+ block
    strides = (block[0]* A.strides[0], block[1]* A.strides[1])+ A.strides
    return ast(A, shape= shape, strides= strides)


def ssim(img1, img2, C1=0.01**2, C2=0.03**2):

    bimg1 = block_view(img1, (4,4))
    bimg2 = block_view(img2, (4,4))
    s1  = numpy.sum(bimg1, (-1, -2))
    s2  = numpy.sum(bimg2, (-1, -2))
    ss  = numpy.sum(bimg1*bimg1, (-1, -2)) + numpy.sum(bimg2*bimg2, (-1, -2))
    s12 = numpy.sum(bimg1*bimg2, (-1, -2))

    vari = ss - s1*s1 - s2*s2
    covar = s12 - s1*s2

    ssim_map =  (2*s1*s2 + C1) * (2*covar + C2) / ((s1*s1 + s2*s2 + C1) * (vari + C2))
    return numpy.mean(ssim_map)

# FIXME there seems to be a problem with this code
def ssim_exact(img1, img2, sd=1.5, C1=0.01**2, C2=0.03**2):

    mu1 = gaussian_filter(img1, sd)
    mu2 = gaussian_filter(img2, sd)
    mu1_sq = mu1 * mu1
    mu2_sq = mu2 * mu2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = gaussian_filter(img1 * img1, sd) - mu1_sq
    sigma2_sq = gaussian_filter(img2 * img2, sd) - mu2_sq
    sigma12 = gaussian_filter(img1 * img2, sd) - mu1_mu2

    ssim_num = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2))

    ssim_den = ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    ssim_map = ssim_num / ssim_den
    return numpy.mean(ssim_map)

In [ ]:
data = []
for i in range(100):
    img1 = cv2.imread(path+'test/frame_real'+str(i)+'.png')
    img2 = imresize(img1, (1, 1))
    img2 = imresize(img2, (384, 384))
    d = ssim_exact(img1/25, img2)
    data.append(d)
    print(d)